### Import mlflow client

In [9]:
from mlflow.tracking import MlflowClient
import pickle

client = MlflowClient()


### Take the best run

In [11]:
# Choose best run
best_run_id = "103f9338648647898c90f9986d0ceac0"
best_run = client.get_run(best_run_id)

# Extract feature columns logged in this run
feature_columns = best_run.data.params["features"]
print("Feature columns of best model:", feature_columns)

# Save feature columns for FastAPI / registry
with open("artifacts/best_model_features.pkl", "wb") as f:
    pickle.dump(feature_columns, f)

# Log them as artifact in MLflow
client.log_artifact(best_run_id, "artifacts/best_model_features.pkl", artifact_path="model_artifacts")

Feature columns of best model: ['surface', 'surface_squared', 'rooms', 'rooms_per_surface', 'governorate_Ben Arous', 'governorate_Gabes', 'governorate_Gafsa', 'governorate_Kairouan', 'governorate_Kasserine', 'governorate_Mahdia', 'governorate_Manouba', 'governorate_Mednine', 'governorate_Monastir', 'governorate_Nabeul', 'governorate_Sfax', 'governorate_Sousse', 'governorate_Tozeur', 'governorate_Tunis', 'governorate_Zaghouan', 'property_type_House']


### Register the best model

In [ ]:


# Register model in the Model Registry
model_name = "HousePriceModel"

# Create the registered model 
try:
    client.create_registered_model(model_name)
except:
    print("Model already exists, skipping creation.")

# Create a new version for this run
version = client.create_model_version(
    name=model_name,
    source=f"runs:/{best_run_id}/model",
    run_id=best_run_id
)

print(f"Registered model version: {version.version}")


Model already exists, skipping creation.
Registered model version: 3


### Promote the version to production 

In [ ]:
client.transition_model_version_stage(
    name=model_name,
    version=version.version,
    stage="Production"
)

print(f"Model version {version.version} is now in Production")




Model version 3 is now in Production


/tmp/ipykernel_108692/2596384108.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(
